In [3]:
#2번. 당진-서버간 duet연결
import syft as sy
duet1 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > CONNECTED!


In [4]:
#4번. 울산-서버간 duet연결
duet2 = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!


Exception in callback Transaction.__retry()
handle: <TimerHandle when=10910.593 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...c\xb2'\xcas2"), ('192.168.10.26', 49362))>



♫♫♫ > CONNECTED!


In [5]:
#9번
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

In [6]:
#10번. 사용데이터의 전처리를 합니다.

# 당진

#사용할 데이터(기상요소)를 불러와줍니다.
dangjin_obs_data_path = r'C:dangjin_obs_data.csv'
dangjin_obs_data = pd.read_csv(dangjin_obs_data_path)
#-----------------------------------------------------------------------------------------------

#데이터프레임에 누락된 6회분의 시간대를 nan값으로 채워줍니다.
dangjin_obs_data.loc[3490.1]=[129,'서산','2018-07-24 11:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
dangjin_obs_data.loc[3490.2]=[129,'서산','2018-07-24 12:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
dangjin_obs_data.loc[3490.3]=[129,'서산','2018-07-24 13:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
dangjin_obs_data.loc[3490.4]=[129,'서산','2018-07-24 14:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
dangjin_obs_data.loc[3490.5]=[129,'서산','2018-07-24 15:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
dangjin_obs_data.loc[3490.6]=[129,'서산','2018-07-24 16:00',np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
#-----------------------------------------------------------------------------------------------

#6개의 행이 새로 추가된 데이터프레임의 인덱스를 정수화해줍니다
dangjin_obs_data = dangjin_obs_data.sort_index() 
dangjin_obs_data = dangjin_obs_data.reset_index(drop = True)
#-----------------------------------------------------------------------------------------------

#데이터프레임에 누락된 부분을 선형보간법으로 채워줍니다.
new_dangjin_obs_data=dangjin_obs_data.interpolate(method='values',limit_direction='backward')
#-----------------------------------------------------------------------------------------------

#기상요소를 정규화하고, noise가 추가된 새로운 데이터 셋도 생성합니다.
mu = 0
ratio=15

tem_dangjin = new_dangjin_obs_data['기온(°C)'].values
wv_dangjin = new_dangjin_obs_data['풍속(m/s)'].values
wd_dangjin = new_dangjin_obs_data['풍향(16방위)'].values
humi_dangjin = new_dangjin_obs_data['습도(%)'].values
cloud_dangjin = new_dangjin_obs_data['전운량(10분위)'].values

sigma_tem_dangjin = np.max(tem_dangjin)*(ratio/100)
sigma_wv_dangjin = np.max(wv_dangjin)*(ratio/100)
sigma_wd_dangjin = np.max(wd_dangjin)*(ratio/100)
sigma_humi_dangjin = np.max(humi_dangjin)*(ratio/100)
sigma_cloud_dangjin = np.max(cloud_dangjin)*(ratio/100)

tem_noise_dangjin = np.random.normal(mu, sigma_tem_dangjin, tem_dangjin.shape) 
wv_noise_dangjin = np.random.normal(mu, sigma_wv_dangjin, wv_dangjin.shape)  
wd_noise_dangjin = np.random.normal(mu, sigma_wd_dangjin, wd_dangjin.shape)  
humi_noise_dangjin = np.random.normal(mu, sigma_humi_dangjin, humi_dangjin.shape)  
cloud_noise_dangjin = np.random.normal(mu, sigma_cloud_dangjin, cloud_dangjin.shape)  
#-----------------------------------------------------------------------------------------------
data_tem_noise_dangjin = np.reshape(np.array(tem_dangjin + tem_noise_dangjin),(1068,24))/np.max(tem_dangjin)
data_wv_noise_dangjin = np.reshape(np.array(wv_dangjin + wv_noise_dangjin),(1068,24))/np.max(wv_dangjin)
data_wd_noise_dangjin = np.reshape(np.array(wd_dangjin + wd_noise_dangjin),(1068,24))/np.max(wd_dangjin)
data_humi_noise_dangjin = np.reshape(np.array(humi_dangjin + humi_noise_dangjin),(1068,24))/np.max(humi_dangjin)
data_cloud_noise_dangjin = np.reshape(np.array(cloud_dangjin + cloud_noise_dangjin),(1068,24))/np.max(cloud_dangjin)

#-----------------------------------------------------------------------------------------------
data_tem_dangjin = np.reshape(tem_dangjin,(1068,24))/np.max(tem_dangjin)
data_wv_dangjin = np.reshape(wv_dangjin,(1068,24))/np.max(wv_dangjin)
data_wd_dangjin = np.reshape(wd_dangjin,(1068,24))/np.max(wd_dangjin)
data_humi_dangjin = np.reshape(humi_dangjin,(1068,24))/np.max(humi_dangjin)
data_cloud_dangjin = np.reshape(cloud_dangjin,(1068,24))/np.max(cloud_dangjin)

Input_dangjin = np.concatenate((data_tem_dangjin,data_wv_dangjin,data_wd_dangjin, data_humi_dangjin, data_cloud_dangjin ), axis=1)
Input_noise_dangjin = np.concatenate((data_tem_noise_dangjin,data_wv_noise_dangjin,data_wd_noise_dangjin, data_humi_noise_dangjin, data_cloud_noise_dangjin ), axis=1)

# 사용데이터를 트레이닝,발리데이션,테스트 셋으로 나눕니다.
train_len=848
valid_len=110

train_input_dangjin = Input_dangjin[:train_len,:]
valid_input_dangjin = Input_dangjin[train_len:train_len+valid_len,:]
test_input_dangjin = Input_noise_dangjin[train_len+valid_len:,:]

#-----------------------------------------------------------------------------------------------

# 울산

#사용할 데이터(기상요소)를 불러와줍니다.
ulsan_obs_data_path = r'C:ulsan_obs_data.csv'
ulsan_obs_data = pd.read_csv(ulsan_obs_data_path)
#-----------------------------------------------------------------------------------------------

#데이터프레임에 누락된 부분을 선형보간법으로 채워줍니다.
new_ulsan_obs_data=ulsan_obs_data.interpolate(method='values',limit_direction='backward')
#-----------------------------------------------------------------------------------------------

#기상요소를 정규화하고, noise가 추가된 새로운 데이터 셋도 생성합니다.
tem_ulsan = new_ulsan_obs_data['기온(°C)'].values
wv_ulsan = new_ulsan_obs_data['풍속(m/s)'].values
wd_ulsan = new_ulsan_obs_data['풍향(16방위)'].values
humi_ulsan = new_ulsan_obs_data['습도(%)'].values
cloud_ulsan = new_ulsan_obs_data['전운량(10분위)'].values

sigma_tem_ulsan = np.max(tem_ulsan)*(ratio/100)
sigma_wv_ulsan = np.max(wv_ulsan)*(ratio/100)
sigma_wd_ulsan = np.max(wd_ulsan)*(ratio/100)
sigma_humi_ulsan = np.max(humi_ulsan)*(ratio/100)
sigma_cloud_ulsan = np.max(cloud_ulsan)*(ratio/100)

tem_noise_ulsan = np.random.normal(mu, sigma_tem_ulsan, tem_ulsan.shape)  # creating a noise with the same dimension as the dataset
wv_noise_ulsan = np.random.normal(mu, sigma_wv_ulsan, wv_ulsan.shape)  # creating a noise with the same dimension as the dataset
wd_noise_ulsan = np.random.normal(mu, sigma_wd_ulsan, wd_ulsan.shape)  # creating a noise with the same dimension as the dataset
humi_noise_ulsan = np.random.normal(mu, sigma_humi_ulsan, humi_ulsan.shape)  # creating a noise with the same dimension as the dataset
cloud_noise_ulsan = np.random.normal(mu, sigma_cloud_ulsan, cloud_ulsan.shape)  # creating a noise with the same dimension as the dataset
#-----------------------------------------------------------------------------------------------
data_tem_noise_ulsan = np.reshape(np.array(tem_ulsan + tem_noise_ulsan),(1068,24))/np.max(tem_ulsan)
data_wv_noise_ulsan = np.reshape(np.array(wv_ulsan + wv_noise_ulsan),(1068,24))/np.max(wv_ulsan)
data_wd_noise_ulsan = np.reshape(np.array(wd_ulsan + wd_noise_ulsan),(1068,24))/np.max(wd_ulsan)
data_humi_noise_ulsan = np.reshape(np.array(humi_ulsan + humi_noise_ulsan),(1068,24))/np.max(humi_ulsan)
data_cloud_noise_ulsan = np.reshape(np.array(cloud_ulsan + cloud_noise_ulsan),(1068,24))/np.max(cloud_ulsan)

#-----------------------------------------------------------------------------------------------
data_tem_ulsan = np.reshape(tem_ulsan,(1068,24))/np.max(tem_ulsan)
data_wv_ulsan = np.reshape(wv_ulsan,(1068,24))/np.max(wv_ulsan)
data_wd_ulsan = np.reshape(wd_ulsan,(1068,24))/np.max(wd_ulsan)
data_humi_ulsan = np.reshape(humi_ulsan,(1068,24))/np.max(humi_ulsan)
data_cloud_ulsan = np.reshape(cloud_ulsan,(1068,24))/np.max(cloud_ulsan)

Input_ulsan = np.concatenate((data_tem_ulsan,data_wv_ulsan,data_wd_ulsan, data_humi_ulsan, data_cloud_ulsan ), axis=1)
Input_noise_ulsan = np.concatenate((data_tem_noise_ulsan,data_wv_noise_ulsan,data_wd_noise_ulsan, data_humi_noise_ulsan, data_cloud_noise_ulsan ), axis=1)

#사용데이터를 트레이닝,발리데이션,테스트 셋으로 나눕니다.

train_input_ulsan = Input_ulsan[:train_len,:]
valid_input_ulsan = Input_ulsan[train_len:train_len+valid_len,:]
test_input_ulsan = Input_noise_ulsan[train_len+valid_len:,:]
#-----------------------------------------------------------------------------------------------





In [7]:
#11번. 두 클라이언트가 pysyft에 올린 데이터를 포인터형태로 불러옵니다.
train_out_ptr_dangjin=duet1.store[0]
valid_out_ptr_dangjin=duet1.store[1]
test_out_ptr_dangjin=duet1.store[2]

train_out_ptr_ulsan=duet2.store[0]
valid_out_ptr_ulsan=duet2.store[1]
test_out_ptr_ulsan=duet2.store[2]

In [8]:
# 12번. 학습에 사용할 모델을 작성합니다.
in_size        = 120
out_size       = 24
train_len      = 848
hidden_size    = 256
batch_size     = 32
learning_rate  = 1e-4

total_epoch    = 100
print_interval = 50

class MLP(sy.Module):
    def __init__(self,torch_ref):
        super(MLP, self).__init__(torch_ref=torch_ref)
        self.fc_in  = self.torch_ref.nn.Linear(in_size, hidden_size)
        self.fc_h1  = self.torch_ref.nn.Linear(hidden_size, hidden_size)
        self.fc_h2  = self.torch_ref.nn.Linear(hidden_size, hidden_size)
        self.fc_h3  = self.torch_ref.nn.Linear(hidden_size, hidden_size)
        self.fc_out = self.torch_ref.nn.Linear(hidden_size, out_size)

        self.torch_ref.torch.nn.init.xavier_uniform_(self.fc_in.weight)
        self.torch_ref.torch.nn.init.xavier_uniform_(self.fc_h1.weight)
        self.torch_ref.torch.nn.init.xavier_uniform_(self.fc_h2.weight)
        self.torch_ref.torch.nn.init.xavier_uniform_(self.fc_h3.weight)
        self.torch_ref.torch.nn.init.xavier_uniform_(self.fc_out.weight)

    def forward(self, x):
        x = self.torch_ref.nn.functional.relu(self.fc_in(x))
        x = self.torch_ref.nn.functional.relu(self.fc_h1(x))
        x = self.torch_ref.nn.functional.relu(self.fc_h2(x))
        x = self.torch_ref.nn.functional.relu(self.fc_h3(x))
        out = self.torch_ref.nn.functional.relu(self.fc_out(x))

        return out


In [9]:
import random
import pdb
def local_train(epoch, model, torch_ref, optim, train_input_ptr, train_output_ptr, batch_size):
    
    losses = []
    length = train_input_ptr.shape[0]
    index_list = torch.randperm(length)
    criterion = torch_ref.nn.L1Loss()
    
    before = 0
    model.train()
    
    for batch_idx in range(1,length//batch_size+1):
        
        batch = batch_size * batch_idx
        if batch >= length:
            batch = length -1
            
        selected_idx = index_list[before:batch]
        
        optim.zero_grad()
      
        data = torch_ref.FloatTensor(train_input_ptr[selected_idx,:])
        target = train_output_ptr[selected_idx,:]

        out = model(data)
        
        loss = criterion(out, target)
        loss = loss/torch_ref.mean(target)

        loss.backward()
        
        optim.step()
        
        loss_item = loss.item()
        
        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        #print("Epoch", batch_idx, "loss", loss_value)
        
        losses.append(loss_value)
        before = batch
        
    loss_avg = np.mean(losses)
    
    return loss_avg, losses

In [10]:
def local_valid(epoch, model, torch_ref, optim, valid_input_ptr, valid_output_ptr, batch_size):
    
    losses = []
    length = valid_input_ptr.shape[0]
    index_list = torch.randperm(length)
    criterion = torch_ref.nn.L1Loss()
    before = 0
    model.eval()
    
    with torch.no_grad():
        for batch_idx in range(1,length//batch_size+1):

            batch = batch_size * batch_idx
            if batch >= length:
                batch = length -1

            selected_idx = index_list[before:batch]

            data = torch_ref.FloatTensor(valid_input_ptr[selected_idx,:])
            target = valid_output_ptr[selected_idx,:]

            out = model(data)
            
            loss = criterion(out, target)
            loss = loss/torch_ref.mean(target)
       
    
            loss_item = loss.item()

            loss_value = loss_item.get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5,
            )

            #print("Epoch", batch_idx, "loss", loss_value)

            losses.append(loss_value)
            before = batch

    loss_avg = np.mean(losses)
        
    return loss_avg, losses

In [11]:
def test(epoch, model, torch_ref,test_input_ptr, test_output_ptr):
    
    losses = []
    length = test_input_ptr.shape[0]
    index_list = torch.randperm(length)
    criterion = torch_ref.nn.L1Loss()

    model.eval()
    
    with torch.no_grad():


        data = torch_ref.FloatTensor(test_input_ptr)
        target = test_output_ptr

        out = model(data)

        loss = criterion(out, target)
        loss = loss/torch_ref.mean(target)

        loss_item = loss.item()

        loss_value = loss_item.get(
            reason="To evaluate training progress",
            request_block=True,
            timeout_secs=5,
        )

        #print("Epoch", batch_idx, "loss", loss_value)

        losses.append(loss_value)

    loss_avg = np.mean(losses)
        
    return loss_avg, losses

In [12]:
from collections import OrderedDict
def federated_traing(local_train, epochs,duet1,duet2, 
                     train_input1, train_output1, 
                     train_input2, train_output2,
                     valid_input1, valid_output1, 
                     valid_input2, valid_output2,
                     test_input1, test_input2, 
                     test_output1, test_output2,
                     batch_size,option):

    global_model = MLP(torch)
    
    remote_model1 = global_model.send(duet1)  #당진
    remote_torch1 = duet1.torch
    params1 = remote_model1.parameters()
    optim1 = remote_torch1.optim.Adam(params=params1, lr=1e-4)
    
    remote_model2 = global_model.send(duet2)  #울산
    remote_torch2 = duet2.torch
    params2 = remote_model2.parameters()
    optim2 = remote_torch2.optim.Adam(params=params2, lr=1e-4)
    
    train_loss1_list = []
    valid_loss1_list = []
    
    train_loss2_list = []
    valid_loss2_list = []
    
    for epoch in range(1,epochs+1):
        print("-----------Start Epoch",epoch,"-----------")
        
        losses1, T_loss1_list = local_train(epoch, remote_model1, remote_torch1, optim1, train_input1, train_output1, 8)
        valid_loss1,V_loss1_list  = local_valid(epoch, remote_model1, remote_torch1, optim1, valid_input1, valid_output1, 8)
        
        print("Epoch",epoch, "Train Loss1 = " ,losses1)
        print("Epoch",epoch, "Valid Loss1 = " ,valid_loss1)
        
        train_loss1_list.extend(T_loss1_list)
        valid_loss1_list.extend(V_loss1_list)

        
        losses2, T_loss2_list = local_train(epoch, remote_model2, remote_torch2, optim2, train_input2, train_output2,8)
        valid_loss2, V_loss2_list  = local_valid(epoch, remote_model2, remote_torch2, optim2, valid_input2, valid_output2, 8)
        
        print("Epoch",epoch, "Train Loss2 = " ,losses2)
        print("Epoch",epoch, "Valid Loss2 = " ,valid_loss2)
        
        train_loss2_list.extend(T_loss2_list)
        valid_loss2_list.extend(V_loss2_list)
        
        del T_loss1_list, T_loss2_list, V_loss1_list, V_loss2_list
        
        if option == "FL":
            if epoch % 4 == 0:
                remote_model1_updates = remote_model1.get(request_block=True).state_dict()
                remote_model2_updates = remote_model2.get(request_block=True).state_dict()

                avg_updates = OrderedDict()
                print("-----------Updating Parameters-----------")

                for weight in remote_model1_updates.keys():
                    avg_updates[weight] = (remote_model1_updates[weight] + remote_model2_updates[weight]) / 2

                updated_model = MLP(torch)
                updated_model.load_state_dict(avg_updates)

                print("-----------Updating Clients-----------")
                remote_model1 = updated_model.send(duet1) 
                remote_model2 = updated_model.send(duet2) 
                print("-----------Done-----------")

                params1 = remote_model1.parameters()
                optim1 = remote_torch1.optim.Adam(params=params1, lr=1e-4)

                params2 = remote_model2.parameters()
                optim2 = remote_torch2.optim.Adam(params=params2, lr=1e-4)


            
                
                
    test_losses1, test_loss1_list = test(epoch, remote_model1, remote_torch1,test_input1, test_output2)
    print("-----",epoch, "Test Loss1 = " ,test_losses1)
    
    test_losses2, test_loss2_list = test(epoch, remote_model2, remote_torch2,test_input2, test_output2)       
    print("-----",epoch, "Test Loss2 = " ,test_losses2)
          
    return train_loss1_list, valid_loss1_list, train_loss2_list, valid_loss2_list

In [13]:
import torch

In [14]:
epochs = 9
batch_size = 8

train_loss1_list, valid_loss1_list, train_loss2_list, valid_loss2_list = federated_traing(local_train,epochs ,duet1,duet2, 
                                                                                          train_input_dangjin, train_out_ptr_dangjin,
                                                                                          train_input_ulsan, train_out_ptr_ulsan,
                                                                                          valid_input_dangjin, valid_out_ptr_dangjin,
                                                                                          valid_input_ulsan, valid_out_ptr_ulsan,
                                                                                          test_input_dangjin, test_out_ptr_dangjin,
                                                                                          test_input_ulsan, test_out_ptr_ulsan,
                                                                                          batch_size,'none')

-----------Start Epoch 1 -----------
Epoch 1 Train Loss1 =  0.46718284417435807
Epoch 1 Valid Loss1 =  0.4689490840985225
Epoch 1 Train Loss2 =  0.48074214090153855
Epoch 1 Valid Loss2 =  0.48506843470610106
-----------Start Epoch 2 -----------
Epoch 2 Train Loss1 =  0.293997548661142
Epoch 2 Valid Loss1 =  0.39589530114944166


Exception in callback Transaction.__retry()
handle: <TimerHandle when=11249.359 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...xc2\xf0\xe94'), ('192.168.10.26', 49362))>


Epoch 2 Train Loss2 =  0.31585228358799555
Epoch 2 Valid Loss2 =  0.47112088937025803
-----------Start Epoch 3 -----------


Exception in callback Transaction.__retry()
handle: <TimerHandle when=11345.453 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...{\xe5pN-\xbb'), ('192.168.10.26', 64513))>


Epoch 3 Train Loss1 =  0.26457991352621113
Epoch 3 Valid Loss1 =  0.3692247821734502
Epoch 3 Train Loss2 =  0.28924849053036494
Epoch 3 Valid Loss2 =  0.4940522863314702
-----------Start Epoch 4 -----------
Epoch 4 Train Loss1 =  0.24763044321312094
Epoch 4 Valid Loss1 =  0.38875911786006045
Epoch 4 Train Loss2 =  0.27072175999857345
Epoch 4 Valid Loss2 =  0.5068415999412537
-----------Start Epoch 5 -----------
Epoch 5 Train Loss1 =  0.2346271515454886
Epoch 5 Valid Loss1 =  0.3369313386770395


Exception in callback Transaction.__retry()
handle: <TimerHandle when=11908.437 Transaction.__retry()>
Traceback (most recent call last):
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aioice\stun.py", line 306, in __retry
    self.__future.set_exception(TransactionTimeout())
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\futures.py", line 270, in set_exception
    raise exceptions.InvalidStateError(f'{self._state}: {self!r}')
asyncio.exceptions.InvalidStateError: FINISHED: <Future finished result=(Message(messa...04GD\xa2\x0f'), ('192.168.10.26', 49362))>


Epoch 5 Train Loss2 =  0.2583086704308132
Epoch 5 Valid Loss2 =  0.4541304398041505
-----------Start Epoch 6 -----------
Epoch 6 Train Loss1 =  0.22578004958494655
Epoch 6 Valid Loss1 =  0.3447674478475864


Task exception was never retrieved
future: <Task finished name='Task-99699' coro=<RTCSctpTransport._transmit() done, defined at C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py:1501> exception=ConnectionError('Cannot send encrypted data, not connected')>
Traceback (most recent call last):
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\asyncio\tasks.py", line 256, in __step
    result = coro.send(None)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1535, in _transmit
    await self._send_chunk(chunk)
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcsctptransport.py", line 1335, in _send_chunk
    await self.__transport._send_data(
  File "C:\Users\dhkim\Anaconda3\envs\pysyft\lib\site-packages\aiortc\rtcdtlstransport.py", line 659, in _send_data
    raise ConnectionError("Cannot send encrypted data, not connected")
ConnectionError: Cannot send encrypted data, not connected


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_loss2_list)
plt.title("ulsan train loss")
plt.show()

In [ ]:
, valid_loss1_list, train_loss2_list, valid_loss2_list